# Hypsometry: how much of the first km coast is below 5 meters? 

This case study demonstrates the use of cloud-native workflows, combining data-proximate computing with cloud-optimized data exposed in a Spatio-Temporal Asset Collection to facilitate coastal analytics at a planetary scale. Specifically, we integrate the GCTS with DeltaDTM, a novel digital terrain model (Pronk et al., 2024), to determine the percentage of the world's first kilometer of coastal zone that is lower than 5 meters. For more information, please refer to Calkoen et al., Enabling Coastal Analytics at Planetary Scale (2024). 

In [ ]:
import sys

from coastpy.utils.config import configure_instance

instance_type = configure_instance()

from odc.stac import configure_rio

configure_rio(cloud_defaults=True)

import logging
import os
import pathlib
import time

import colorcet as cc
import dask

dask.config.set({"dataframe.query-planning": False})

import dask.dataframe as dd
import dask_geopandas
import fsspec
import geopandas as gpd
import hvplot.pandas
import hvplot.xarray
import numpy as np
import pandas as pd
import pystac
import pystac_client
import rioxarray
import shapely
import xarray as xr
from dotenv import load_dotenv
from geopandas.array import GeometryDtype
from ipyleaflet import Map, basemaps

from coastpy.geo.quadtiles import make_mercantiles

load_dotenv(override=True)

sas_token = os.getenv("AZURE_STORAGE_SAS_TOKEN")
storage_account_name = os.getenv("AZURE_STORAGE_ACCOUNT")
storage_options = {"account_name": storage_account_name, "sas_token": sas_token}

LOWER_THAN = (
    5  # threshold in meter to compute "xx of the 1-km coastal zone is <LOWER_THAN> m."
)
H3_LEVEL = 7  # h3 hexagonal-grid zoom-level to aggregate the results
MERCANTILES_LEVEL = 4

COMPUTE_GCTS_LANDWARD = False
COMPUTE_GCTS_ELEVATION = False
COMPUTE_H3_ELEVATION_STATISTICS = False
COMPUTE_GLOBAL_LT_STATISTIC = False

GCTS_LANDWARD_CONTAINER = "az://public/coastal-analytics/gcts-2000m-landward.parquet"
# NOTE: before we stored the results here, keep for a while as ref
GCTS_ELEVATION_CONTAINER = "az://public/coastal-analytics/gcts-2000m-elevation.parquet"
# NOTE: Next iteration we will store it here and extract the profiles including the tr_name
# GCTS_ELEVATION_CONTAINER = "az://coastal-transect-repository/deltadtm-elevation.parquet"
H3_ELEVATION_CONTAINER = (
    f"az://public/coastal-analytics/h3-l{H3_LEVEL}-pct-lt-{LOWER_THAN}m.parquet"
)

## Connect to the CoCliCo STAC

In [ ]:
from coastpy.io.utils import read_items_extent

# Read the CoCliCo STAC catalog
coclico_catalog = pystac.Catalog.from_file(
    "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
)

# Read the spatial footprint of the transect partitions.
gcts_collection = coclico_catalog.get_child("gcts")
gcts_extents = read_items_extent(gcts_collection, columns=["geometry", "assets"])

# Read the spatial footprint of the DeltaDTM tiles.
ddtm_collection = coclico_catalog.get_child("deltares-delta-dtm")
ddtm_extents = read_items_extent(ddtm_collection, columns=["geometry", "assets"])

## Make a compute cluster

Depending on the instance type that is stored in `instance_type`, launch a Dask Gateway or LocalCluster, 

In [ ]:
from coastpy.utils.dask_utils import create_dask_client

instance_type = configure_instance()
client = create_dask_client(instance_type)
client

## Part 1: Compute the landward part of the cross-shore transects

The cross-shore coastal transects are 2km long, extending 1km in both land and see-ward direction. The statistics are computed over the 1km landward side. 

In [ ]:
if COMPUTE_GCTS_LANDWARD:

    start_time = time.time()

    def map_extract_landward_side(df):
        def extract_landward_side(row):
            p1 = shapely.Point(row.geometry.coords[0])
            p2 = shapely.Point(row.lon, row.lat)
            return shapely.LineString([p1, p2])

        geoms = df.apply(extract_landward_side, axis=1)
        return gpd.GeoDataFrame(df["tr_name"], geometry=geoms, crs=4326)

    gcts_collection = coclico_catalog.get_child("gcts")
    gcts_extents = read_items_extent(gcts_collection, columns=["geometry", "assets"])
    gcts_hrefs = gcts_extents.href.to_list()

    # template GDF that matches what is retunred from map_extract_landward_side
    META = gpd.GeoDataFrame(
        {
            "tr_name": gpd.GeoSeries([], dtype=str),
            "geometry": gpd.GeoSeries([], dtype=GeometryDtype()),
        }
    )

    transects = dask_geopandas.read_parquet(
        gcts_hrefs,
        storage_options=storage_options,
        columns=["tr_name", "geometry", "lon", "lat"],
    )

    transects = transects.map_partitions(map_extract_landward_side, meta=META)
    transects.to_parquet(GCTS_LANDWARD_CONTAINER, storage_options=storage_options)
    elapsed_time = time.time() - start_time
    print(f"Time (H:M:S): {time.strftime('%H:%M:%S', time.gmtime(elapsed_time))}")
    client.restart()

## Part 2: Extract elevation data per transect

1) First filter out the files that have already been processed
2) Extract the elevation data per transect for each Deltares DeltaDTM tile 

In [ ]:
if COMPUTE_GCTS_ELEVATION:

    from dask.distributed import TimeoutError

    start_time = time.time()

    def to_out_href(href, out_prefix):
        return out_prefix + "/" + href.split("_")[-1].split(".")[0] + ".parquet"

    @dask.delayed
    def extract_by_geometry(href, transects, storage_options):
        """Extract elevation raster data by transect LineString geometry in columnar format."""

        da = rioxarray.open_rasterio(href, chunks={}, lock=False).squeeze().drop("band")

        bbox = da.rio.bounds()
        bbox = gpd.GeoDataFrame(geometry=[shapely.box(*bbox)], crs=4326)

        transects = gpd.overlay(transects, bbox)

        if transects.empty:
            return

        da = da.where(da != da.rio.nodata, np.nan)
        da = da.rio.write_nodata(np.nan)

        # TODO: ensure that tr_name is tracked so that we can use the elevation data later at a transect level
        clipped = da.rio.clip(transects.geometry.to_list()).rename("band_data")

        df = (
            clipped.drop(["spatial_ref"])
            .to_dataframe()
            .dropna()
            .reset_index()
            .rename(columns={"x": "lng", "y": "lat"})
            .rename(columns={"band_data": "z"})[["lng", "lat", "z"]]
        )

        out_href = to_out_href(href, GCTS_ELEVATION_CONTAINER)

        with fsspec.open(out_href, "wb", **storage_options) as f:
            df.to_parquet(f)

    ddtm_collection = coclico_catalog.get_child("deltares-delta-dtm")
    ddtm_extents = read_items_extent(ddtm_collection, columns=["geometry", "assets"])

    tiles = make_mercantiles(MERCANTILES_LEVEL)

    ddtm_extents = (
        gpd.sjoin(ddtm_extents, tiles)
        .drop(columns="index_right")
        .sample(frac=1)
        .drop_duplicates("href", keep="first")
        .sort_index()
    )
    ddtm_extents["out_href"] = ddtm_extents.href.map(
        lambda href: to_out_href(href, GCTS_ELEVATION_CONTAINER)
    )

    fs = fsspec.filesystem("az", **storage_options)
    processed_files = fs.glob(f"{GCTS_ELEVATION_CONTAINER}/*.parquet")
    processed_files = [f"az://{f}" for f in processed_files]

    ddtm_extents = ddtm_extents.loc[~ddtm_extents["out_href"].isin(processed_files)]
    print(f"Number of DeltaDTM tiles to process: {len(ddtm_extents)}")

    for name, gr in ddtm_extents.groupby("quadkey"):
        print(
            f"Start processing quadkey {name}, with total bounds: {gr.total_bounds}, that in total consist of {len(gr)} DeltaDTM tiles."
        )

        roi = gpd.GeoDataFrame(geometry=[shapely.box(*gr.total_bounds)], crs=4326)
        ddtm_hrefs = gr.href.to_list()

        client.restart(wait_for_workers=False)

        try:
            transects = (
                dask_geopandas.read_parquet(
                    GCTS_LANDWARD_CONTAINER, storage_options=storage_options
                )
                .sjoin(roi)
                .drop(columns=["index_right"])
                .compute(timeout="5m")
            )
        except TimeoutError:
            print("Loading transects timed out after 5 minutes.")
            continue

        transects_scattered = client.scatter(transects, broadcast=True)

        tasks = []
        for href in ddtm_hrefs:
            href = f"https://coclico.blob.core.windows.net/{href.strip('az://')}?{sas_token}"
            tasks.append(
                extract_by_geometry(href, transects_scattered, storage_options)
            )

        try:
            dask.compute(
                *tasks, timeout="8m"
            )  # 8 minutes timeout for the whole operation
        except TimeoutError:
            print("The operation timed out after 8 minutes.")
            continue

    elapsed_time = time.time() - start_time
    print(f"Time (H:M:S): {time.strftime('%H:%M:%S', time.gmtime(elapsed_time))}")

## Part 3: Computing statistics

#### Install required dependencies on all workers on the cluster

Please note that this step is only required for Planetary Computer as H3-Pandas is not included in their container. 

In [ ]:
if COMPUTE_H3_ELEVATION_STATISTICS:
    import os

    install_cmd = "mamba install h3pandas -y"

    def worker_setup(dask_worker):
        import os

        os.system(install_cmd)

    if instance_type.name == "MSPC":
        client.register_worker_callbacks(worker_setup)

### Part 3.1: computing statistics by binning into H3 hexagons

In [ ]:
if COMPUTE_H3_ELEVATION_STATISTICS:
    import h3pandas

    start_time = time.time()

    gr_key = f"h3_{H3_LEVEL:02d}"

    DTYPES = {
        f"{gr_key}": "object",
        f"pct_lt_{LOWER_THAN}m": "int32",
        "n_obs": "int32",
        "geometry": GeometryDtype(),
        "lon": "float32",
        "lat": "float32",
    }

    # Create META using DTYPES
    META = pd.DataFrame({col: pd.Series(dtype=dt) for col, dt in DTYPES.items()})

    def lower_than(group, threshold):
        count = (group["z"] < threshold).sum()
        total_count = group["z"].count()
        percentage = (count / total_count) * 100
        return pd.Series(
            [percentage, total_count], index=[f"pct_lt_{threshold}m", "n_obs"]
        )

    def add_h3(df, level, threshold, meta):
        import h3pandas

        start_time = time.time()

        if df.empty:
            # Ensure the empty DataFrame matches the META structure
            return pd.DataFrame([], columns=meta.columns).astype(meta.dtypes)

        df = df.h3.geo_to_h3(level)

        # Convert level to string with leading zeros if necessary
        level_str = f"{level:02d}"  # Ensures two digits
        h3_group_key = f"h3_{level_str}"
        result = (
            df.drop(columns=["lng", "lat"])
            .groupby(h3_group_key)
            .apply(lambda gr: lower_than(gr, threshold))
            .reset_index()
        )
        result = result.set_index(h3_group_key).h3.h3_to_geo_boundary().reset_index()

        points = result.set_index("h3_07").h3.h3_to_geo().geometry
        result["lon"] = points.x.values
        result["lat"] = points.y.values

        # Ensure the result matches the META structure and data types
        result = result.astype(meta.dtypes.to_dict())

        return result

    ddf = dd.read_parquet(
        GCTS_ELEVATION_CONTAINER + "/*.parquet",
        storage_options=storage_options,
    )

    ddf = ddf.map_partitions(
        lambda df: add_h3(df, level=H3_LEVEL, threshold=LOWER_THAN, meta=META),
        meta=META,
    )
    df = ddf.compute().reset_index(drop=True)

    with fsspec.open(H3_ELEVATION_CONTAINER, mode="wb", **storage_options) as f:
        df.to_parquet(f)

    elapsed_time = time.time() - start_time
    print(f"Time (H:M:S): {time.strftime('%H:%M:%S', time.gmtime(elapsed_time))}")

    client.restart()

### Part 3.2: Compute global statistic

In [ ]:
import dask.dataframe as dd

if COMPUTE_GLOBAL_LT_STATISTIC:

    ddf = dd.read_parquet(
        GCTS_ELEVATION_CONTAINER + "/*.parquet", storage_options=storage_options
    )

    pct_lt_5m = ((ddf["z"] < LOWER_THAN).mean() * 100).compute()
    print(
        f"We find that {round(pct_lt_5m)}% is lower than {LOWER_THAN}m in the 1km coastal zone."
    )

## Part 4: Visualization

For the visualization we mostly use Holoviews with a Bokeh backend.

In [ ]:
import cartopy.crs as ccrs
import colorcet as cc
import geopandas as gpd
import geoviews as gv
import h3pandas
import holoviews as hv
import hvplot.pandas
import panel as pn
from bokeh.io import export_svg
from cartopy import crs as ccrs
from holoviews import opts

hv.extension("bokeh")
pn.extension()

### Part 4.1: Read the elevation statistics from cloud object store

In [ ]:
with fsspec.open(H3_ELEVATION_CONTAINER, mode="rb", **storage_options) as f:
    h3_elevation = gpd.read_parquet(f)

### Part 4.2: Aggregate data on lower H3 grid to avoid overplotting

Additionaly we filter out the H3 hexagon's that cross antimerdian boundaries

In [ ]:
def crosses_antimeridian(geom):
    """
    Checks if a geometry crosses the anti-meridian.
    """
    longitudes = [point[0] for point in geom.exterior.coords]
    return any(
        abs(longitudes[i] - longitudes[i - 1]) > 180 for i in range(1, len(longitudes))
    )


def to_h3_agg(df, zoom):
    h3_agg = (
        df[["h3_07", "pct_lt_5m"]]
        .set_index("h3_07")
        .h3.h3_to_parent(zoom)
        .set_index(f"h3_0{zoom}")
        .groupby(f"h3_0{zoom}")
        .mean()
        .h3.h3_to_geo_boundary()
    )

    h3_agg["crosses_antimeridian"] = (
        h3_agg["geometry"].astype(object).apply(crosses_antimeridian)
    )
    h3_agg = h3_agg.loc[h3_agg["crosses_antimeridian"] == False]
    return h3_agg


zoom = 3
h3_agg = to_h3_agg(h3_elevation, zoom)

## Part 4.3: Plot global distribution of coastal land that lower than 5 meters

In [ ]:
global_lt_5m_plot = h3_agg[["pct_lt_5m", "geometry"]].hvplot(
    geo=True,
    cmap=cc.CET_L20[::-1],
    color="pct_lt_5m",
    size=8,
    projection=ccrs.Robinson(),
    features=["land"],
    alpha=0.75,
    colorbar=False,
    width=539,  # Env. modeling fig instructions
    global_extent=True,
)

global_lt_5m_plot.opts(
    hv.opts.Polygons(
        # colorbar_position="left",
        line_color=None,
        colorbar_opts={
            "title": None,
            # "title": "Lower than 5 m (%)",
            # "orientation": "vertical",
            # "width": 15,
            # "height": 400,
            "scale_alpha": 1,
        },
        toolbar="disable",
    )
)

grid = gv.feature.grid(
    projection=ccrs.Robinson(), title="", fill_color="none", color="gray"
)

global_lt_5m_plot = global_lt_5m_plot * grid
global_lt_5m_plot

#### Export as SVG

In [ ]:
H3_GLOBAL_LT_5M_FP = (
    f"az://figures/enabling-coastal-analytics/h3_0{zoom}-global-lt-5m.svg"
)

renderer = hv.renderer("bokeh")
bokeh_figure = renderer.get_plot(global_lt_5m_plot).state
bokeh_figure.output_backend = "svg"
bokeh_figure.background_fill_color = None
bokeh_figure.border_fill_color = None

# outpath = (pathlib.Path("~") / H3_GLOBAL_LT_5M_FP.strip("az://")).expanduser()
# outpath.parent.mkdir(exist_ok=True, parents=True)

# with outpath.open(mode="w") as fp:
#     export_svg(bokeh_figure, filename=fp.name)

#     # Write to cloud storage
#     with fsspec.open(H3_GLOBAL_LT_5M_FP, mode="wb", **storage_options) as cloud_file:
#         with open(fp.name, mode="rb") as fp_read:
#             cloud_file.write(fp_read.read())

### Part 4.4: Hotspot analysis using spatial correlation

In [ ]:
from esda import Moran_Local
from libpysal import weights


def add_lisa(gdf):
    W = weights.Queen.from_dataframe(gdf)
    W.transform = "r"

    # LISA analysis
    lisa = Moran_Local(gdf["pct_lt_5m"].values, W)

    # Identifying significant hotspots
    gdf["lisa_q"] = (
        lisa.q
    )  # The quadrant they belong to; 1 (HH), 2 (LH), 3 (LL), 4 (HL)
    gdf["p_val"] = lisa.p_sim  # P-values for assessing significance
    return gdf


def format_hotspot_df(df):
    def compute_area(df, utm_epsg):
        df = df.to_crs(utm_epsg)
        return df.geometry.area

    with fsspec.open(
        "https://coclico.blob.core.windows.net/public/utm_grid.parquet"
    ) as f:
        utm_grid = gpd.read_parquet(f)

    df = df.dissolve().explode().reset_index(drop=True)

    df["utm_epsg"] = gpd.sjoin(
        df.geometry.representative_point().to_frame("geometry"), utm_grid
    )["epsg"]

    area = (
        df.groupby("utm_epsg")
        .apply(lambda gr: compute_area(gr, gr.name))
        .reset_index()
        .rename(columns={0: "area"})
        .sort_values("level_1")
        .reset_index(drop=True)[["area"]]
    )

    df = df.join(area).sort_values("area", ascending=False)
    return df


h3_agg2 = to_h3_agg(h3_elevation, 5)
h3_agg2 = add_lisa(h3_agg2)

hotspots = h3_agg2[(h3_agg2["lisa_q"] == 1) & (h3_agg2["p_val"] < 0.05)]
hotspots = format_hotspot_df(hotspots)
hotspots = hotspots.iloc[: int(len(hotspots) * 0.025)]

In [ ]:
hotspots_plot = global_lt_5m_plot * hotspots.assign(
    geometry=hotspots.representative_point()
).hvplot.points(
    geo=True,
    size="area",
    color="red",
    scale=0.00003,
    alpha=0.6,
    projection=ccrs.Robinson(),
)

hotspots_plot

#### Export hotspots as SVG

In [ ]:
H3_GLOBAL_LT_5M_HOTSPOTS_FP = (
    f"az://figures/enabling-coastal-analytics/h3_0{zoom}-global-lt-5m-hotspots.svg"
)

renderer = hv.renderer("bokeh")
bokeh_figure = renderer.get_plot(hotspots_plot).state
bokeh_figure.output_backend = "svg"
bokeh_figure.background_fill_color = None
bokeh_figure.border_fill_color = None

# outpath = (pathlib.Path("~") / H3_GLOBAL_LT_5M_HOTSPOTS_FP.strip("az://")).expanduser()
# outpath.parent.mkdir(exist_ok=True, parents=True)

# with outpath.open(mode="w") as fp:
#     export_svg(bokeh_figure, filename=fp.name)

#     # Write to cloud storage
#     with fsspec.open(H3_GLOBAL_LT_5M_FP, mode="wb", **storage_options) as cloud_file:
#         with open(fp.name, mode="rb") as fp_read:
#             cloud_file.write(fp_read.read())

### Part 4:5: Violin plot of the global coastal elevation in the first km

In [ ]:
with fsspec.open(H3_ELEVATION_CONTAINER, mode="rb", **storage_options) as f:
    h3_elevation = gpd.read_parquet(f)

In [ ]:
def width_in_pixels(mm, dpi):
    return int((mm / 25.4) * dpi)


hv.extension("bokeh")

# Assuming h3_elevation is a pandas DataFrame with the column 'pct_lt_5m' containing the data
violin_plot = hv.Violin(h3_elevation[["pct_lt_5m"]], vdims="pct_lt_5m")

# Configure the plot options
violin_plot = violin_plot.opts(
    opts.Violin(
        inner="quartiles",
        cmap=cc.CET_L20,
        # yticks=[(i, f"{i}") for i in np.arange(0, 101, 20)],
        # xticks=None,
        ylabel="",
        # fontsize={"ticks": 9},
        width=width_in_pixels(mm=140 * (1 / 3), dpi=97),
        cut=0,  # clips violin to (0,100)
        border=0,
        padding=0,
        show_frame=False,
        yaxis="right",
    )
)

violin_plot

#### Export to svg

In [ ]:
H3_GLOBAL_DISTRIBUTION_LT_5M = (
    f"az://figures/enabling-coastal-analytics/h3_0{zoom}-global-distribution-lt-5m.svg"
)

outpath = (pathlib.Path("~") / H3_GLOBAL_DISTRIBUTION_LT_5M.strip("az://")).expanduser()
outpath.parent.mkdir(exist_ok=True, parents=True)

renderer = hv.renderer("bokeh")
bokeh_figure = renderer.get_plot(violin_plot).state
bokeh_figure.output_backend = "svg"
# bokeh_figure.background_fill_color = None
# bokeh_figure.border_fill_color = None
# bokeh_figure.outline_line_color = None
# bokeh_figure.outline_line_width = 0


# with outpath.open(mode="w") as fp:
#     export_svg(bokeh_figure, filename=fp.name)

#     # Write to cloud storage
#     with fsspec.open(H3_GLOBAL_LT_5M_FP, mode="wb", **storage_options) as cloud_file:
#         with open(fp.name, mode="rb") as fp_read:
#             cloud_file.write(fp_read.read())

## Part 5: Figure with example of high-resolution data extraction

In [ ]:
import os

import duckdb
import geoviews.tile_sources as gvts
import holoviews as hv
import hvplot.pandas  # Ensures hvplot is available
from bokeh.io import export_png
from holoviews import opts
from holoviews.operation.datashader import dynspread
from ipyleaflet import Map, basemaps
from shapely import wkb

hv.extension("bokeh")
pn.extension()

### Part 5.1 Select a region of interest

In [ ]:
from ipyleaflet import Map, basemaps

m = Map(basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
m.center = 15.825, -95.95
m.zoom = 14
m.layout.height = "725px"
m

In [ ]:
roi = shapely.geometry.box(m.west, m.south, m.east, m.north)
roi = gpd.GeoDataFrame(geometry=[roi], crs=4326)

In [ ]:
GCTS_CONTAINER = gcts_collection.extra_fields["base_url"]

### Part 5.2 Extract the data using DuckDB

We do not have STACs for all data that was produced in this experiment, so we just use DuckDB to efficiently filter the container by predicate pushdown. 

In [ ]:
# Extract the bounding box coordinates from the region of interest (ROI)
minx, miny, maxx, maxy = roi.total_bounds

# Connect to an in-memory DuckDB instance and load necessary extensions
con = duckdb.connect(database=":memory:", read_only=False)
con.execute("INSTALL azure;")
con.execute("LOAD azure;")
con.execute("INSTALL spatial;")
con.execute("LOAD spatial;")

# Create a secret for Azure connection
try:
    con.execute(
        f"""
        CREATE SECRET azure_secret (
        TYPE AZURE,
        CONNECTION_STRING '{os.getenv('CLIENT_AZURE_STORAGE_CONNECTION_STRING')}');
        """
    )
except Exception as e:
    print(f"The secret is likely already available. Exception message:\n\n{e}")

# Define the query to read transect data within the bounding box
transect_query = f"""
    SELECT *
    FROM read_parquet('{GCTS_CONTAINER + "/*.parquet"}')
    WHERE
        bbox.xmin <= {maxx} AND
        bbox.ymin <= {maxy} AND
        bbox.xmax >= {minx} AND
        bbox.ymax >= {miny};
"""

# Execute the transect query and fetch the result as a DataFrame
transect_df = con.execute(transect_query).fetchdf()

# Convert ROI geometry to WKT format
roi_wkt = roi.geometry.to_wkt().iloc[0]

# Create or replace the transects table and fetch the intersecting transects
con.execute(
    f"""
    CREATE OR REPLACE TABLE transects AS 
    SELECT * FROM read_parquet('{GCTS_LANDWARD_CONTAINER + "/*.parquet"}');
    """
)
landward_transects_df = con.execute(
    f"""
    SELECT * 
    FROM transects 
    WHERE ST_Intersects(ST_GeomFromWKB(transects.geometry), ST_GeomFromText('{roi_wkt}'));
    """
).fetchdf()

# Define the query to read elevation data within the bounding box
elevation_query = f"""
    SELECT *
    FROM read_parquet('{GCTS_ELEVATION_CONTAINER + "/*.parquet"}')
    WHERE
        lng <= {maxx} AND
        lat <= {maxy} AND
        lng >= {minx} AND
        lat >= {miny};
"""

# Execute the elevation query and fetch the result as a DataFrame
elevation_df = con.execute(elevation_query).fetchdf()

In [ ]:
transect_gdf = gpd.GeoDataFrame(
    transect_df,
    geometry=transect_df.geometry.apply(lambda x: wkb.loads(bytes(x))),
    crs=4326,
)

landward_transects_gdf = gpd.GeoDataFrame(
    landward_transects_df,
    geometry=landward_transects_df.geometry.apply(lambda x: wkb.loads(bytes(x))),
    crs=4326,
)

elevation_gdf = gpd.GeoDataFrame(
    elevation_df,
    geometry=gpd.points_from_xy(elevation_df.lng, elevation_df.lat, crs=4326),
)

#### Plot the elevation data

In [ ]:
elevation_plot = elevation_gdf.hvplot(
    geo=True, datashade=True, color="z", cmap=cc.CET_L20, colorbar=True, width=800
)
dynspread(elevation_plot, threshold=0.5, max_px=200) * gvts.EsriImagery()

### Part 5.2: Adjust the area of interest

In [ ]:
inner_m = Map(basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
inner_m.center = 15.827, -95.96

inner_m.zoom = 15
inner_m.layout.height = "725px"
inner_m.layout.width = "725px"
inner_m

In [ ]:
inner_roi = shapely.geometry.box(
    inner_m.west, inner_m.south, inner_m.east, inner_m.north
)
inner_roi_gdf = gpd.GeoDataFrame(geometry=[inner_roi], crs=4326)

In [ ]:
transect_gdf_inner = transect_gdf.iloc[
    gpd.sjoin(
        gpd.GeoSeries.from_xy(transect_gdf.lon, transect_gdf.lat, crs=4326).to_frame(
            "geometry"
        ),
        inner_roi_gdf,
    ).index.to_list()
]

landward_transects_gdf_inner = gpd.sjoin(
    landward_transects_gdf, transect_gdf_inner[["geometry"]]
).drop(columns=["index_right"])

# Create a buffer so that we only show elevation data at transects
landward_transect_buffer = landward_transects_gdf_inner.to_crs(
    landward_transects_gdf_inner.estimate_utm_crs()
).buffer(50)

elevation_gdf_inner = (
    gpd.sjoin(
        elevation_gdf.reset_index(),
        landward_transect_buffer.to_crs(4326).to_frame("geometry"),
    )
    .drop_duplicates("index")
    .drop(columns=["index", "index_right"])
)

# transect_gdf_inner_bounds = gpd.GeoDataFrame(
#     geometry=[shapely.geometry.box(*transect_gdf_inner.total_bounds)], crs=4326
# )

# elevation_gdf_inner = gpd.sjoin(elevation_gdf, transect_gdf_inner_bounds).drop(
#     columns=["index_right"]
# )

### Part 5.3 Create the plot

In [ ]:
hr_plot = gvts.EsriImagery() * (
    transect_gdf_inner[["geometry"]].hvplot(
        geo=True,
        width=1200,
        label="GCTS - seaward",
        xlabel="Longitude",
        ylabel="Latitude",
        line_width=4,
        fontscale=3,
    )
    * elevation_gdf_inner.hvplot(
        geo=True,
        cmap=cc.CET_L20,
        color="z",
        colorbar=True,
        clabel="DeltaDTM elevation (m)",
        fontscale=3,
        size=200,
    )
    * landward_transects_gdf_inner.hvplot(
        geo=True,
        color="red",
        alpha=1,
        label="GCTS - landward",
        fontscale=3,
        line_width=4,
    )
)

# Apply opts to customize legends and colorbar
hr_plot.opts(
    hv.opts.Overlay(
        legend_position="bottom_right",  # Position the legend at the bottom right
        toolbar="disable",  # Toolbar position can be adjusted here if needed
    )
)

# Display the final plot
pn.Column(hr_plot).show()

#### Export figure

In [ ]:
H3_HR_LT_5M = (
    f"az://figures/enabling-coastal-analytics/hr-data-extract-barra-de-la-cruz.png"
)

outpath = (pathlib.Path("~") / H3_HR_LT_5M.strip("az://")).expanduser()
outpath.parent.mkdir(exist_ok=True, parents=True)

renderer = hv.renderer("bokeh")
bokeh_figure = renderer.get_plot(hr_plot).state
# bokeh_figure.output_backend = "png"
bokeh_figure.background_fill_color = None
bokeh_figure.border_fill_color = None


# with outpath.open(mode="w") as fp:
#     export_png(bokeh_figure, filename=fp.name)